In [1]:
import pandas as pd
import numpy as np

In [2]:
import datetime as DT

In [3]:
from matplotlib import pyplot as plt

In [4]:
import seaborn as sns

# Lecture des fichiers

In [5]:
file_comptes = pd.read_table("C:/Users/hp/pfedossier/Comptes.txt", sep= "|")

In [18]:
file_comptes['Devise']= file_comptes['Devise'].replace('XOF','MGA', regex=True)

In [ ]:
file_comptes['SECTEURA']

In [ ]:
file_pret = pd.read_table("C:/Users/hp/pfedossier/prets.txt", sep= "|")

In [ ]:
file_pret['DevisePret']= file_pret['DevisePret'].replace('XOF','MGA', regex=True)

In [ ]:
file_pret

In [ ]:
file_pret.info()

In [ ]:
file_clients = pd.read_table("C:/Users/hp/pfedossier/Client.txt", sep= "|")

In [ ]:
file_typepret =pd.read_table("C:/Users/hp/pfedossier/TypePret.txt")

In [ ]:
# extraction des colonnes à partir du fichier clients
info_cli=file_clients[['Client','DateNaissance','Segment','Sexe','Profession','SituationMatrimoniale','Revenus','SECTEURA','Nationalite']]

In [ ]:
file_clients.info()

In [ ]:
#sélection des prets >'2019-01-01'
tes=file_pret.loc[(file_pret['DateDebPret']>"2019-01-01"),:]

In [ ]:
# extraction des colonnes à partir du fichier prets
pret=tes[['Compte','TypePret','DateDebPret','NbImpayesActuel','MtPretInit','NbEcheancePretInit']]

calcul de l'historique

In [ ]:
test=file_pret.loc[(file_pret['DateDebPret']<"2019-01-01")&(file_pret['DateEchPretInit']<"2018-01-01"),:]

In [ ]:
min(test['DateEchPretInit'])

In [ ]:
file_pr=test[['TypePret','Client','DateDebPret']]

In [ ]:
file_pr=file_pr.drop_duplicates()

In [ ]:
#calcul du nmb de pret
file_pr['NbPret']=file_pr.groupby('Client')['TypePret'].transform('size')

In [ ]:
#suppression descolonnes qui vont pas être utilisées
file_pr=file_pr.drop(['TypePret','DateDebPret'],axis=1)

In [ ]:
#aprés calcul du NbPret on va merger les fichiers

file_c=pd.merge(  file_comptes[['Client','Compte']]
                ,file_pr,
                                  on ='Client' )

In [ ]:
file_c=file_c.drop_duplicates()

In [45]:
file= pd.read_table("C:/Users/hp/pfedossier/fichier_aprés_pre_processing",sep='|')

In [59]:
import csv
file.to_csv('fichier_aprés_pre_processing',sep='|')

In [62]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18523 entries, 0 to 18522
Data columns (total 13 columns):
Unnamed: 0               18523 non-null int64
Client                   18523 non-null object
NbJoursCR                18523 non-null int64
NbJoursDB                18523 non-null int64
SoldeCrMoyDev            18523 non-null float64
SoldeDbMoyDev            18523 non-null float64
nbp                      18523 non-null int64
Statut                   18523 non-null int64
SECTEURA                 18523 non-null object
Sexe                     18523 non-null object
SituationMatrimoniale    18523 non-null object
Revenus                  18523 non-null object
age                      18523 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 1.8+ MB


In [56]:
file['SECTEURA']=file['SECTEURA'].replace(['COMMERCE DE GROS'],
                                              'PARTICULIERS NON SALARIES',regex=True)

Calcul de l'âge

In [ ]:
#Convertir le type de date de naissance de object à datetime
info_cli['DateNaissance']= pd.to_datetime(info_cli['DateNaissance']) 

In [ ]:
#Calcul de l'âge des clients à partir de la date de naissance
now = pd.Timestamp('now')
info_cli['age'] = (now - info_cli['DateNaissance']).astype('<m8[Y]')    # 3

In [ ]:
info_cli.drop_duplicates()

In [ ]:
info_cli=info_cli.drop(['DateNaissance'],axis=1)

In [ ]:
#fichier type pret ne contient pas de nom de colonne
file_typepret.columns=['TypePret','NamePret']
file_typepret

In [ ]:
#emerge le fichier de pret avec le fichier qui contient les types de pret
pret_typret= pd.merge(  file_typepret ,pret,
                                  on ='TypePret' )

In [ ]:
pret_typret=pret_typret.drop_duplicates()

In [ ]:
#Aprés jointure on supprime la colonne 
pret_typret=pret_typret.drop(['TypePret'],axis=1)
pret_typret

In [ ]:
#le compte a des valeurs NbImpayesActuel répétées 
pret_typret['NbImpayesActuel']=pret_typret.groupby(['Compte'])['NbImpayesActuel'].transform(max)

In [ ]:
pret_typret['Compte'].nunique()

In [ ]:
# merge des fichiers info_cli & train_aux qui contient le nb de pret pour chaque client le nb d'impayes actuel

file_d= pd.merge(  pret_typret ,file_c,
                                  on ='Compte' )

In [ ]:
file_d=file_d.drop_duplicates()

In [ ]:
# merge des fichiers file_d & file_ qui contient le nb de pret pour chaque client le nb d'impayes actuel
file_f= pd.merge( file_d ,info_cli,
                                  on ='Client' )

In [ ]:
file_f=file_f.drop(['Client','Segment'],axis=1)

In [ ]:
file_f['Statut']=file_f["NbImpayesActuel"].map(lambda x: 1 if x >= 3
                                                 else 0 if x < 3
                                                 else -1)

In [ ]:
file_f['Nationalite']= file_f['Nationalite'].replace('SN','MG', regex=True)

In [ ]:
file_f['SECTEURA'].value_counts()

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['AUTRES COMMERCE DE GROS','AUTRES ACTIVITES ANNEXES','comptes interne','SERVICES DE REPARATION',
                                             'IMPORT DISTR CARB COMBUS'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['TRANSPORTS PAR VOIE TERRESTRE','AUTRES AFFS IMMOBILIERES'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['PECHE','PROD ENERG HYDRO ET BARR'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['DISTR COURANT ELECTRIQUE','AUTRES PARTICULIERS NON SALARIES'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['RESTAUR ET DEBITS BOISS'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['SERVICES DOMESTIQUES'],'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['COMMERCE DE DETAIL'],
                                              'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(['SERVICES NON SPECIFIES','AUTRES COMMERCES'],
                                              'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(["COMM GENE D'IMPORT EXPORT"],
                                              'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f['SECTEURA']=file_f['SECTEURA'].replace(["PARTICULIERS NON-SALARIES"],
                                              'PARTICULIERS NON SALARIES',regex=True)

In [ ]:
file_f=file_f.drop(['Profession'],axis=1)

# 1.Préparation des données: valeurs manquantes

In [ ]:
print(file_f.isnull().sum())

In [ ]:
# la plupart des valeurs Nan dans la colonne revenus sont aussi manquantes dans la colonne profession
#remplasse les valeurs Nan par "sans revenus fixe"
file_f['Revenus']=file_f['Revenus'].replace(np.nan, 'Nondeclare')

In [ ]:
#les rows avec NaN dans la colonne SECTEURA vont être supprimés
file_f=file_f.dropna(subset=['SECTEURA'])

In [ ]:
file_f['age'] = file_f['age'].fillna((file_f['age'].median()))

In [ ]:
#Remplir les valeurs manquantes par la mediane
file_f['NbEcheancePretInit']=file_f['NbEcheancePretInit'].fillna((file_f['NbEcheancePretInit'].median()))

In [ ]:
file_f.fillna(method='ffill', inplace = True)

In [ ]:
f=file_f[['Compte','SECTEURA']]

In [ ]:
#print(file_f.loc[(file_f['SituationMatrimoniale']!='C')&(file_f['SituationMatrimoniale']!='M')])

In [ ]:
file_f['MtPretInit']=file_f['MtPretInit'].replace(',','.', regex=True)

In [ ]:
file_f['age']=file_f['age'].astype('int')

In [ ]:
file_f['NbEcheancePretInit']=file_f['NbEcheancePretInit'].apply(pd.to_numeric)

In [ ]:
file_f['MtPretInit']=file_f['MtPretInit'].apply(pd.to_numeric)

In [ ]:
file_f['MtPretInit']=file_f['MtPretInit'].astype(int)

In [ ]:
file_f=file_f.drop(['DateDebPret','NbImpayesActuel'],axis=1)

In [ ]:
file_f['NbEcheancePretInit']=file_f['NbEcheancePretInit'].astype(int)

In [ ]:
file_f["Nationalite"].replace(["CI", "BJ",'ML','BF','FR','GN','NE','CG','CF','GA','CM ','MA','MR','GW','TD','RW','SD','CD', 'GM',
                               
       'XX','ES','BI','NG','IN','AE','EG','TG','CM'], "Other", inplace=True)

In [ ]:
file_f['Nationalite'].value_counts()

Nettoyage de la variable Revenus

In [ ]:
file_f['Revenus'].value_counts()

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['de 35,000 a 50,000'],'A1',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['de 50,000 a 100,000'],'A1',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['de 100,000 a 200,000'],'A2',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['150000'],'A2',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['200000'],'A2',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['de 200,000 a 350,000'],'A3',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['349000'],'A3',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['250000'],'A3',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['de 350,000 a 500,000'],'A4',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['500000'],'A3',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['SANS REVENU FIXE'],'A0',regex=True)

In [ ]:
file_f['Revenus']=file_f['Revenus'].replace(['plus de 500,000'],'A5',regex=True)

In [ ]:
file_f['MtPretInit'].value_counts()

In [ ]:
 pd . qcut ( file_f[ 'MtPretInit' ],  q = 5 ) 

In [ ]:
file_f['SituationMatrimoniale'].value_counts()

In [ ]:
file_f['SituationMatrimoniale']=file_f['SituationMatrimoniale'].replace('N','M',regex=True)

In [ ]:
file_f['SituationMatrimoniale']=file_f['SituationMatrimoniale'].replace('V','D',regex=True)

In [ ]:
file_f['NamePret'].value_counts()

# 2. Exploratory Data Analysis

In [ ]:
f= pd.read_table("C:/Users/hp/pfedossier/fichier",sep='|')

In [14]:
f['Statut'].value_counts()

0    17957
1      695
Name: Statut, dtype: int64

,Unnamed: 0,Unnamed: 0.1,Client,NbJoursCR,NbJoursDB,SoldeCrMoyDev,SoldeDbMoyDev,nbp,Statut,SECTEURA,Sexe,SituationMatrimoniale,Revenus,age
0,18,4570,BABCDC,58,31,"99483,00","-96416,42",21,0,PARTICULIERS NON SALARIES,M,C,SANS REVENU FIXE,42.0
1,23,5818,BABCEF,48,30,"98322,58","-8203,16",15,0,PARTICULIERS SALARIES,F,C,"de 350,000 a 500,000",46.0
2,42,11370,BABDFG,31,0,",00",",00",3,0,PARTICULIERS NON SALARIES,M,M,SANS REVENU FIXE,55.0
3,76,16445,BAIDFI,52,31,"98125,13","-985995,80",41,0,PARTICULIERS NON SALARIES,F,C,"plus de 500,000",32.0
4,83,18512,BAIEAB,54,31,"9288,19","-9814,30",15,0,PARTICULIERS NON SALARIES,F,C,SANS REVENU FIXE,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18647,35693,4819849,JHEBJH,31,23,"9696,45","-28493,43",3,0,PARTICULIERS NON SALARIES,M,M,"de 35,000 a 50,000",27.0
18648,35694,4819863,JHEHDG,31,27,"9353,13","-603,84",3,0,PARTICULIERS NON SALARIES,M,M,"de 50,000 a 100,000",34.0
18649,35695,4819877,EDFEHB,48,13,"9031,16","-92,32",2,0,PARTICULIERS NON SALARIES,M,M,"de 50,000 a 100,000",59.0
18650,35699,4819991,EHECED,37,7,"81,60","-545,71",3,0,PARTICULIERS NON SALARIES,M,C,"de 100,000 a 200,000",32.0


In [ ]:
file['Revenus'].value_counts()

In [ ]:
labels = ['non défaut','défaut']
classes = pd.value_counts(file_f['Statut'], sort = True)
classes.plot(kind = 'bar', rot=0)
plt.title("Transaction class distribution")
plt.xticks(range(2), labels)
plt.xlabel("prêt")
plt.ylabel("Frequency")

In [ ]:
plt.scatter(file_f['Revenus'], file_f['age'],c='blue', alpha=0.5)
plt.xlabel("Revenus du personne")
plt.ylabel("age")
plt.show()

In [ ]:
file_f.describe()

# Détection des outliers

In [24]:
pd.crosstab(file['Revenus'], file['Statut'])

Statut,0,1
Revenus,,
0,841,104
SANS REVENU FIXE,2030,143
"de 100,000 a 200,000",6529,192
"de 35,000 a 50,000",1142,70
"de 350,000 a 500,000",1321,33
"de 50,000 a 100,000",4335,113
"plus de 500,000",1759,40


In [ ]:
max(file_f['age'])

In [ ]:
plt.scatter(file['Revenus'],file_f['age'], c='blue', alpha=0.5)
plt.xlabel('Revenus')
plt.ylabel('age')
plt.show()

In [34]:
file['SECTEURA'].value_counts()

PARTICULIERS NON SALARIES    11471
PARTICULIERS SALARIES         6993
ORGANISATIONS INTERNATS         19
SECTEURS NON IDENTIF             1
Name: SECTEURA, dtype: int64

In [31]:
print(pd.crosstab(file['Statut'],file['Revenus'],values=file['age']
                  ,aggfunc=['max','min']))

          max                                                           \
Revenus     0 SANS REVENU FIXE de 100,000 a 200,000 de 35,000 a 50,000   
Statut                                                                   
0        70.0             76.0                 83.0               72.0   
1        42.0             73.0                 69.0               62.0   

                                                                   min  \
Revenus de 350,000 a 500,000 de 50,000 a 100,000 plus de 500,000     0   
Statut                                                                   
0                       91.0                82.0            80.0  21.0   
1                       58.0                74.0            61.0  24.0   

                                                                  \
Revenus SANS REVENU FIXE de 100,000 a 200,000 de 35,000 a 50,000   
Statut                                                             
0                   21.0                 20.0         

On remarque qu'il existe des outliers 

In [28]:
#suppression des outliers
indices = file[file['age'] <18].index
file.drop(indices, inplace=True)

In [ ]:
file_f.to_pickle('fichier_final')

In [ ]:
file_f['Revenus'].value_counts()

In [ ]:
#aprés suppression des outliers
print(pd.crosstab(file_f['Statut'],file_f['Revenus'],values=file_f['age']
                  ,aggfunc=['max','min']))

In [ ]:
#Aprés suppression des outliers
plt.scatter(file_f['Revenus'],file_f['age'], c='blue', alpha=0.5)
plt.xlabel('Revenus')
plt.ylabel('age')
plt.show()

In [ ]:
plt.figure()
sns.catplot(x='SituationMatrimoniale',data=file_f, col='Statut', kind="count").set_axis_labels("Statut Civil","count")
plt.show()

In [ ]:
file_f["NamePret"].value_counts(normalize=True).plot(kind='pie')

In [ ]:
data=file_f[['age','NbEcheancePretInit','Revenus','MtPretInit']]
data['Age_Bins'] = pd.qcut(data['age'], 5, 
                           labels=['Age_lt_31', 'Age_31_to_35', 'Age_35_to_40', 'Age_40_to_46', 'Age_gt_46'])

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=data['Age_Bins'], y=data['MtPretInit'], palette="Reds_r")
plt.xlabel('\nage' , fontsize=15, color='#c0392b')
plt.ylabel("Montant de prêt initial\n", fontsize=15, color='#c0392b')
plt.title("Distribution du type du prêt par âge\n", fontsize=18, color='#e74c3c')
plt.xticks(rotation= 45)
plt.tight_layout()

# Etude de la Multicolinéarité 

In [ ]:
# Set Up Inputs and Target
numerical = ['MtPretInit', 'NbEcheancePretInit', 'age','NbPret']
categorical = ['NamePret','Sexe',  'SituationMatrimoniale', 'Revenus', 'SECTEURA', 'Nationalite']
target = 'target'

In [ ]:
print(numerical)

In [ ]:
print(categorical)

In [ ]:
target='Statut'

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,5))
sns.set(font_scale=1)
sns.heatmap(file_f[numerical].corr(), cmap="Blues", annot=True, fmt=".2f")
plt.show()

Comme le montre la matrice de corrélation, aucune des caractéristiques ne présente une forte corrélation (coeff>0,7). 
Par conséquent, nous n'avons pas besoin d'exclure des caractéristiques de l'ensemble de données.


# Set up Weight of Evidence Transform

In [ ]:
file=pd.read_pickle("C:/Users/hp/pfedossier/fichier_final")

In [ ]:
file_f['age'].describe()

we Keep in mind the values for the 25%, 50% and 75% percentiles as we look at using qcut directly.

In [ ]:
file['Age_Bins'] = pd.qcut(file['age'], 5, 
                           labels=['Age_lt_31', 'Age_31_to_35', 'Age_35_to_40', 'Age_40_to_46', 'Age_gt_46'])

In [ ]:
 pd . qcut ( file[ 'NbEcheancePretInit' ],  q = 3 ) 

In [ ]:
max(file['NbEcheancePretInit'])

In [ ]:
file['NbEchPret_Bins'] = pd.qcut(file['NbEcheancePretInit'], 3, 
                           labels=['NbEchP_lt_36', 'NbEchP_36_to_60', 'NbEchP_60_to_240'])

In [ ]:
file['MtPretInit_Bins'] = pd.qcut(file['MtPretInit'], 5, 
                           labels=['MtPr_1t_968000', 'MtPr__968000to_2000000', 
                                   'MtPr__2000000to_3000000', 'MtPr_3000000_to_46', 'MtPr_5000000_to_2000000000'])

In [ ]:
file.drop ([ 'MtPretInit' , 'age'], axis = 1 , inplace = True )

In [ ]:
file.drop([ 'NbEcheancePretInit'], axis = 1 , inplace = True )

In [ ]:
file_f

In [ ]:
df = pd.get_dummies(file, columns=['NamePret', 'Sexe', 
                                 'SituationMatrimoniale', 'Revenus', 'SECTEURA', 'Nationalite'])

In [ ]:
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(file[feature].nunique()):
        val = list(file[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': file[file[feature] == val].count()[feature],
            'Good': file[(file[feature] == val) & (file[target] == 0)].count()[feature],
            'Bad': file[(file[feature] == val) & (file[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

Affichage des valeurs IV 

In [ ]:
for col in file.columns:
    if col == 'Statut': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(file, col, 'Statut')
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

In [ ]:
file_f.info()